## Github Actions and ACT


Table of Contents
* **Triggers and Matrices**
* Variables and Secrets
* Authentication and Deployment
* Local Development with ACT

### Terminology

job - a shell script or action
action - 
workflow - one or more jobs
runner - [ubuntu-lastest]
events - [push, PR, manual trigger]

### Triggers

```yaml
on:
  push:
    branches:
    - main
    - 'releases/**'
  pull_request:
    branches:
    - main
    - 'releases/**'
```

### Variables

```yaml
env:
  TAG_NAME: "tag_${{ github.sha }}_${{ github.run_attempt }}"
```

```yaml
run: echo ${{ github.repository }} ${{ env.TAG_NAME }}
```

### Combining Them

```yaml
name: Example 1 Workflow
on:
  push:
  workflow_dispatch:
env:
  TAG_NAME: "tag/${{ github.sha }}/${{ github.run_attempt }}"
jobs:
  example1:
    name: Example 1 Job
    runs-on: ubuntu-latest
    steps:
    - name: Example 1 Action
      run: echo ${{ github.repository }} ${{ env.TAG_NAME }}
```

### Running the Workflow

In [ ]:
!gh workflow run example1.yaml --ref main

In [ ]:
!gh run list --workflow example1.yaml --limit 1

### Message Passing


In [ ]:
```yaml
- name: Format Tag Name String
  id: format_tag_name
  run: |
    tag_name="${{ env.TAG_NAME }}"
    formatted_tag_name=${tag_name//\//-} # replace forward slash with -
    echo "::set-output name=FORMATTED_TAG_NAME::$formatted_tag_name"
```

```yaml
- name: Echo Formatted Tag Name
  run: echo ${{ steps.format_tag_name.outputs.FORMATTED_TAG_NAME }}
```

### External Workflows

```yaml
sync_argo_cd:
if: needs.ci.outputs.branch_type == 'main'
    needs: ci
    uses: evan-certn/actions-ttt/.github/workflows/argocd_deploy.yaml@main
```


```yaml
- name: Echo Formatted Tag Name
  run: echo ${{ steps.format_tag_name.outputs.FORMATTED_TAG_NAME }}
```

### External Workflows

```yaml
sync_argo_cd:
if: needs.ci.outputs.branch_type == 'main'
    needs: ci
    uses: evan-certn/actions-ttt/.github/workflows/argocd_deploy.yaml@main
```